In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [9]:
with open('SiteNumber.txt', 'r') as file:
    lines = file.readlines()
numbers = []
for line in lines:
    number = re.findall(r'\d+', line)
    numbers.append(number)
df_numbers = pd.DataFrame(numbers, columns=['Number'])
lines = [line.strip() for line in lines]
df_result = pd.DataFrame({'Original': lines})
df_result = pd.concat([df_result, df_numbers], axis=1)

print(df_result)

                Original           Number
0          USGS-02336000         02336000
1          USGS-02336300         02336300
2          USGS-02336526         02336526
3          USGS-02336728         02336728
4          USGS-02336410         02336410
5          USGS-02336313         02336313
6          USGS-02336658         02336658
7          USGS-02335880         02335880
8          USGS-02203655         02203655
9         USGS-023365218        023365218
10         USGS-02203603         02203603
11        USGS-023362773        023362773
12         USGS-02336360         02336360
13  USGS-340326084223201  340326084223201
14         USGS-02203679         02203679
15         USGS-02336517         02336517
16         USGS-02336503         02336503
17  USGS-333402084350501  333402084350501
18         USGS-02335450         02335450
19         USGS-02336504         02336504
20  USGS-334608084254201  334608084254201
21        USGS-023365037        023365037
22  USGS-340215084178701  34021508

In [11]:
for index, row in df_result.iterrows():
    site_number = row['Number']
    url = f"https://waterdata.usgs.gov/nwis/inventory/?site_no={site_number}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        dd_tags = soup.find_all('dd')
        for dd in dd_tags:
            if "Latitude" in dd.text and "Longitude" in dd.text:
                split_text = dd.text.split()
                
                latitude_string = ''.join(char for char in split_text[1] if not char.isalpha() or char.isdigit())
                degrees = int(latitude_string.split('°')[0])
                minutes = int(latitude_string.split('°')[1].split('\'')[0])
                seconds = float(latitude_string.split('°')[1].split('\'')[1].split('\"')[0])
                latitude = degrees + minutes / 60 + seconds / 3600
                
                longitude_string = ''.join(char for char in split_text[3] if not char.isalpha() or char.isdigit())
                degrees = int(longitude_string.split('°')[0])
                minutes = int(longitude_string.split('°')[1].split('\'')[0])
                seconds = float(longitude_string.split('°')[1].split('\'')[1].split('\"')[0])
                longitude = degrees + minutes / 60 + seconds / 3600
                
                df_result.at[index, 'LatitudeMeasure'] = latitude
                df_result.at[index, 'LongitudeMeasure'] = longitude
                break
    else:
        print(f"Failed to fetch data for site number: {site_number}")
df_result

Latitude  33°51'33",   Longitude  84°27'16"   NAD83
Latitude  33°49'13.1",   Longitude  84°24'27.5"   NAD83
Latitude  33°47'40.1",   Longitude  84°28'27.1"   NAD83
Latitude  33°44'36.7",   Longitude  84°34'05.4"   NAD83
Latitude  33°50'17.6",   Longitude  84°26'21.4"   NAD83
Latitude  33°49'17.9",   Longitude  84°26'19.1"   NAD83
Latitude  33°44'20.6",   Longitude  84°28'44.8"   NAD83
Latitude  33°54'07.1",   Longitude  84°26'34.9"   NAD83
Latitude  33°40'44.1",   Longitude  84°21'29.3"   NAD83
Latitude  33°47'34",   Longitude  84°28'08"   NAD83
Latitude  33°41'02",   Longitude  84°24'55"   NAD83
Latitude  33°47'46",   Longitude  84°22'12"   NAD83
Latitude  33°52'09",   Longitude  84°22'44"   NAD83
Latitude  34°03'11.26",   Longitude  84°22'18.32"   NAD83
Latitude  33°43'05",   Longitude  84°21'14"   NAD27
Latitude  33°46'33.5",   Longitude  84°26'26.2"   NAD83
Latitude  33°45'36.8",   Longitude  84°25'41.3"   NAD83
Latitude  33°34'32.02",   Longitude  84°35'59.05"   NAD83
Latitude  33

,Original,Number,LatitudeMeasure,LongitudeMeasure
0,USGS-02336000,02336000,33.859167,84.454444
1,USGS-02336300,02336300,33.820306,84.407639
2,USGS-02336526,02336526,33.794472,84.474194
3,USGS-02336728,02336728,33.743528,84.568167
4,USGS-02336410,02336410,33.838222,84.439278
5,USGS-02336313,02336313,33.821639,84.438639
6,USGS-02336658,02336658,33.739056,84.479111
7,USGS-02335880,02335880,33.901972,84.443028
8,USGS-02203655,02203655,33.678917,84.358139
9,USGS-023365218,023365218,33.792778,84.468889


In [12]:
df_result.to_csv('SiteCoordinate.csv', index=False)